In [1]:
"""Keras implementation of SSD."""

from tensorflow.keras import models,layers
from tf_ssd_layers import PriorBox

In [2]:
input_shape=(300,300,3)
num_classes= 21

In [3]:
input_layer = layers.Input(shape=(300,300,3))

#Block 1
conv1_1 = layers.Conv2D(64, (3, 3), name='conv1_1', padding='same', activation='relu')(input_layer)
conv1_2 = layers.Conv2D(64, (3, 3), name='conv1_2', padding='same', activation='relu')(conv1_1)
pool1 =   layers.MaxPooling2D(name='pool1', pool_size=(2, 2), strides=(2, 2), padding='same', )(conv1_2)

#Block 2
conv2_1 = layers.Conv2D(128, (3, 3), name='conv2_1', padding='same', activation='relu')(pool1)
conv2_2 = layers.Conv2D(128, (3, 3), name='conv2_2', padding='same', activation='relu')(conv2_1)
pool2 =   layers.MaxPooling2D(name='pool2', pool_size=(2, 2), strides=(2, 2), padding='same')(conv2_2)

# Block 3
conv3_1 = layers.Conv2D(256, (3, 3), name='conv3_1', padding='same', activation='relu')(pool2)
conv3_2 = layers.Conv2D(256, (3, 3), name='conv3_2', padding='same', activation='relu')(conv3_1)
conv3_3 = layers.Conv2D(256, (3, 3), name='conv3_3', padding='same', activation='relu')(conv3_2)
pool3 = layers.MaxPooling2D(name='pool3', pool_size=(2, 2), strides=(2, 2), padding='same')(conv3_3)

# Block 4
conv4_1 = layers.Conv2D(512, (3, 3), name='conv4_1', padding='same', activation='relu')(pool3)
conv4_2 = layers.Conv2D(512, (3, 3), name='conv4_2', padding='same', activation='relu')(conv4_1)
conv4_3 = layers.Conv2D(512, (3, 3), name='conv4_3', padding='same', activation='relu')(conv4_2)
pool4 = layers.MaxPooling2D(name='pool4', pool_size=(2, 2), strides=(2, 2), padding='same')(conv4_3)

# Block 5
conv5_1 = layers.Conv2D(512, (3, 3), name='conv5_1', padding='same', activation='relu')(pool4)
conv5_2 = layers.Conv2D(512, (3, 3), name='conv5_2', padding='same', activation='relu')(conv5_1)
conv5_3 = layers.Conv2D(512, (3, 3), name='conv5_3', padding='same', activation='relu')(conv5_2)
pool5 = layers.MaxPooling2D(name='pool5', pool_size=(3, 3), strides=(1, 1), padding='same')(conv5_3)

model = models.Model(inputs=input_layer, outputs=pool5)
from tensorflow.keras.utils import plot_model
plot_model(model)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.


In [15]:

# FC6
fc6 = layers.Conv2D(1024, (3, 3), name='fc6', dilation_rate=(6, 6), padding='same', activation='relu')(pool5)

# FC7
fc7 = layers.Conv2D(1024, (1, 1), name='fc7', padding='same', activation='relu')(fc6)

# Block 6
conv6_1 = layers.Conv2D(256, (1, 1), name='conv6_1', padding='same', activation='relu')(fc7)
conv6_2 = layers.Conv2D(512, (3, 3), name='conv6_2', strides=(2, 2), padding='same', activation='relu')(conv6_1)

# Block 7
conv7_1 = layers.Conv2D(128, (1, 1), name='conv7_1', padding='same', activation='relu')(conv6_2)
conv7_1z = layers.ZeroPadding2D(name='conv7_1z')(conv7_1)
conv7_2 = layers.Conv2D(256, (3, 3), name='conv7_2', padding='valid', strides=(2, 2), activation='relu')(conv7_1z)

# Block 8
conv8_1 = layers.Conv2D(128, (1, 1), name='conv8_1', padding='same', activation='relu')(conv7_2)
conv8_2 = layers.Conv2D(256, (3, 3), name='conv8_2', padding='same', strides=(2, 2), activation='relu')(conv8_1)

# Last Pool
pool6 = layers.GlobalAveragePooling2D(name='pool6')(conv8_2)

# Prediction from conv4_3
num_priors = 3
img_size = (input_shape[1], input_shape[0])
name = 'conv4_3_norm_mbox_conf'
conv4_3_norm = layers.LayerNormalization(name='conv4_3_norm')(conv4_3) 
conv4_3_norm_mbox_loc = layers.Conv2D(num_priors * 4, (3, 3), name='conv4_3_norm_mbox_loc', padding='same')(conv4_3_norm)
conv4_3_norm_mbox_loc_flat = layers.Flatten(name='conv4_3_norm_mbox_loc_flat')(conv4_3_norm_mbox_loc)
conv4_3_norm_mbox_conf = layers.Conv2D(num_priors * num_classes, (3, 3), name=name, padding='same')(conv4_3_norm)
conv4_3_norm_mbox_conf_flat = layers.Flatten(name='conv4_3_norm_mbox_conf_flat')(conv4_3_norm_mbox_conf)
conv4_3_norm_mbox_priorbox = PriorBox(img_size, 30.0, name='conv4_3_norm_mbox_priorbox', aspect_ratios=[2], variances=[0.1, 0.1, 0.2, 0.2])(conv4_3_norm)

# Prediction from fc7
num_priors = 6
name = 'fc7_mbox_conf'
fc7_mbox_conf = layers.Conv2D(num_priors * num_classes, (3, 3), padding='same', name=name)(fc7)
fc7_mbox_conf_flat = layers.Flatten(name='fc7_mbox_conf_flat')(fc7_mbox_conf)
fc7_mbox_loc = layers.Conv2D(num_priors * 4, (3, 3), name='fc7_mbox_loc', padding='same')(fc7)
fc7_mbox_loc_flat = layers.Flatten(name='fc7_mbox_loc_flat')(fc7_mbox_loc)
fc7_mbox_priorbox = PriorBox(img_size, 60.0, name='fc7_mbox_priorbox', max_size=114.0, aspect_ratios=[2, 3], variances=[0.1, 0.1, 0.2, 0.2])(fc7)

# Prediction from conv6_2
num_priors = 6
name = 'conv6_2_mbox_conf'
conv6_2_mbox_conf = layers.Conv2D(num_priors * num_classes, (3, 3), padding='same', name=name)(conv6_2)
conv6_2_mbox_conf_flat = layers.Flatten(name='conv6_2_mbox_conf_flat')(conv6_2_mbox_conf)
conv6_2_mbox_loc = layers.Conv2D(num_priors * 4, (3, 3,), name='conv6_2_mbox_loc', padding='same')(conv6_2)
conv6_2_mbox_loc_flat = layers.Flatten(name='conv6_2_mbox_loc_flat')(conv6_2_mbox_loc)
conv6_2_mbox_priorbox = PriorBox(img_size, 114.0, max_size=168.0, aspect_ratios=[2, 3], variances=[0.1, 0.1, 0.2, 0.2], name='conv6_2_mbox_priorbox')(conv6_2)

# Prediction from conv7_2
num_priors = 6
name = 'conv7_2_mbox_conf'
conv7_2_mbox_conf = layers.Conv2D(num_priors * num_classes, (3, 3), padding='same', name=name)(conv7_2)
conv7_2_mbox_conf_flat = layers.Flatten(name='conv7_2_mbox_conf_flat')(conv7_2_mbox_conf)
conv7_2_mbox_loc = layers.Conv2D(num_priors * 4, (3, 3), padding='same', name='conv7_2_mbox_loc')(conv7_2)
conv7_2_mbox_loc_flat = layers.Flatten(name='conv7_2_mbox_loc_flat')(conv7_2_mbox_loc)
conv7_2_mbox_priorbox = PriorBox(img_size, 168.0, max_size=222.0, aspect_ratios=[2, 3], variances=[0.1, 0.1, 0.2, 0.2], name='conv7_2_mbox_priorbox')(conv7_2)

# Prediction from conv8_2
num_priors = 6
name = 'conv8_2_mbox_conf'
conv8_2_mbox_conf = layers.Conv2D(num_priors * num_classes, (3, 3), padding='same', name=name)(conv8_2)
conv8_2_mbox_conf_flat = layers.Flatten(name='conv8_2_mbox_conf_flat')(conv8_2_mbox_conf)
conv8_2_mbox_loc = layers.Conv2D(num_priors * 4, (3, 3), padding='same', name='conv8_2_mbox_loc')(conv8_2)
conv8_2_mbox_loc_flat = layers.Flatten(name='conv8_2_mbox_loc_flat')(conv8_2_mbox_loc)
conv8_2_mbox_priorbox = PriorBox(img_size, 222.0, max_size=276.0, aspect_ratios=[2, 3],
                                     variances=[0.1, 0.1, 0.2, 0.2], name='conv8_2_mbox_priorbox')(conv8_2)

# Prediction from pool6
num_priors = 6
name = 'pool6_mbox_conf_flat'
target_shape = (1, 1, 256)
pool6_mbox_loc_flat = layers.Dense(num_priors * 4, name='pool6_mbox_loc_flat')(pool6)
pool6_mbox_conf_flat = layers.Dense(num_priors * num_classes, name=name)(pool6)
pool6_reshaped = layers.Reshape(target_shape, name='pool6_reshaped')(pool6)
pool6_mbox_priorbox = PriorBox(img_size, 276.0, max_size=330.0, aspect_ratios=[2, 3],
                                   variances=[0.1, 0.1, 0.2, 0.2], name='pool6_mbox_priorbox')(pool6_reshaped)

# Gather all predictions
mbox_loc = layers.concatenate([conv4_3_norm_mbox_loc_flat,
                            fc7_mbox_loc_flat,
                            conv6_2_mbox_loc_flat,
                            conv7_2_mbox_loc_flat,
                            conv8_2_mbox_loc_flat,
                            pool6_mbox_loc_flat],
                           axis=1,
                           name='mbox_loc')
mbox_conf = layers.concatenate([conv4_3_norm_mbox_conf_flat,
                             fc7_mbox_conf_flat,
                             conv6_2_mbox_conf_flat,
                             conv7_2_mbox_conf_flat,
                             conv8_2_mbox_conf_flat,
                             pool6_mbox_conf_flat],
                            axis=1,
                            name='mbox_conf')
mbox_priorbox = layers.concatenate([conv4_3_norm_mbox_priorbox,
                                 fc7_mbox_priorbox,
                                 conv6_2_mbox_priorbox,
                                 conv7_2_mbox_priorbox,
                                 conv8_2_mbox_priorbox,
                                 pool6_mbox_priorbox],
                                axis=1,
                                name='mbox_priorbox')
if hasattr(mbox_loc, '_shape_val'):
        num_boxes = mbox_loc._shape_val[-1] // 4
elif hasattr(mbox_loc, 'int_shape'):
        num_boxes = K.int_shape(mbox_loc)[-1] // 4

mbox_loc = layers.Reshape((num_boxes, 4),
                       name='mbox_loc_final')(mbox_loc)
mbox_conf = layers.Reshape((num_boxes, num_classes),
                        name='mbox_conf_logits')(mbox_conf)
mbox_conf = layers.Activation('softmax',
                           name='mbox_conf_final')(mbox_conf)

predictions = layers.concatenate([mbox_loc,
                               mbox_conf,
                               mbox_priorbox],
                              axis=2,
                              name='predictions')

    



In [16]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_6 (InputLayer)            [(None, 300, 300, 3) 0                                            
__________________________________________________________________________________________________
conv1_1 (Conv2D)                (None, 300, 300, 64) 1792        input_6[0][0]                    
__________________________________________________________________________________________________
conv1_2 (Conv2D)                (None, 300, 300, 64) 36928       conv1_1[0][0]                    
__________________________________________________________________________________________________
pool1 (MaxPooling2D)            (None, 150, 150, 64) 0           conv1_2[0][0]                    
______________________________________________________________________________________________

In [21]:
from tensorflow.keras.utils import plot_model
plot_model(model, to_file="test.png", show_shapes=True)

Failed to import pydot. You must install pydot and graphviz for `pydotprint` to work.
